In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy

In [2]:
!ls '../../ppc/scripts/'

autotune_12000.sh
autotune_eval_10000.sh
autotune_eval_12000_seed.sh
autotune_eval_12000.sh
autotune_eval_8000.sh
autotune_eval_zifa_A.sh
autotune_eval_zifa_B.sh
autotune_eval_zifa_C_new.sh
autotune_eval_zifa_C.sh
autotune_eval_zifa_D_new.sh
autotune_eval_zifa_D.sh
autotune_eval_zifa_F_new.sh
autotune_eval_zifa_F.sh
autotune_eval_zifa_F_sos10.sh
autotune_eval_zifa_G_new.sh
autotune_eval_zifa_G.sh
autotune_eval_zifa_H.sh
autotune_eval_zifa_I.sh
autotune.sh
autotune_zi_12000.sh
autotune_zifa_new.sh
autotune_zifa.sh
autotune_zi.sh
aws_sos
brain_small
brain_small_old
brain_small.zip
catchup_new.txt
catchup.txt
check_hemato.py
cortex
cortex_bis
cortex_old
cortex.zip
data
hemato
hemato_old
hemato_zinb_1200_50_results.json
hemato_zinb_1200_50_results_new.json
hemato.zip
klein_dataset
klein_dataset_nb_1200_50_results.json
klein_dataset_zinb_1200_50_results.json
log_poisson_nb_dataset_10000
log_poisson_nb_dataset_10000_old1
log_poisson_nb_dataset_10000_sim1
log_poisson_nb_dataset_12000
log_pois

In [5]:
data_path = '../../ppc/scripts/'
dataset_name = 'hemato'
data_files = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
data_names = []
for data_file in data_files:
    data_name = data_file.split('/')[-1].split('_')[0]
    data_names.append(data_name)
    print(data_file)


../../ppc/scripts/hemato/nb_hemato.p
../../ppc/scripts/hemato/zinb_hemato.p


In [6]:
data_names

['nb', 'zinb']

In [8]:
dfs = []
for data_name, f in zip(data_names, data_files):
    if data_name != 'zifa_full':
        
        #my_df = pd.read_csv(f, sep='\t')
        my_df = pd.read_pickle(f)
        my_df.loc[:, 'model'] = data_name
        dfs.append(my_df)
        print(data_name, f)
#         display(my_df)
df = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/hemato/nb_hemato.p
zinb ../../ppc/scripts/hemato/zinb_hemato.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [10]:


n_to_keep = 50

metrics = ['ll_ll', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']

h1_hypothesis = ['greater', 'greater', 'greater', 'greater']

df_nb = df.loc[df.model=='nb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
df_zinb = df.loc[df.model=='zinb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]

df_nb_2 = df_nb.copy()
df_nb_2['model'] = 'nb'
df_zinb_2 = df_zinb.copy()
df_zinb_2['model'] = 'zinb'
df_new = pd.concat([df_nb_2, df_zinb_2])


from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby, other_df):
    my_df = gby[metrics]
    assert my_df.shape[1] == len(metrics)
    assert other_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
        assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
        #_, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
        try:
            _, pval = scipy.stats.mannwhitneyu(np.sort(other_df.values[:, idx])[:n_to_keep],
                                   np.sort(my_df.values[:, idx])[:n_to_keep][::-1],
                                   use_continuity=True,
                                   alternative=alternative)
        except ValueError:
            
            pval = 0.5

        pvals.append(pval)
    return np.array(pvals)


pvals_against_zinb = df_new.groupby('model').apply(get_pvals, other_df=df_zinb)
pvals_against_zinb = (pvals_against_zinb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})


pvals_against_nb = df_new.groupby('model').apply(get_pvals, other_df=df_nb)
pvals_against_nb = (pvals_against_nb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})


In [14]:
def get_summary(gby):
    res = {}
    res['median'] = gby.median()
    res['pvals_against_nb'] = pvals_against_nb[gby.name]
    res['pvals_against_zi'] = pvals_against_zinb[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df_new.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
#     print(val)
    style = ['background-color: yellow' if v < 0.05 else '' for v in val]
    return style

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
style = df_summary.style.apply(my_styler, subset=['pvals_against_nb', 'pvals_against_zi'], axis=1)
print(dataset_name)
style
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)
# style

hemato
